In [1]:
import os
import pandas as pd

### There you can change parameters, which is controlling GA

In [2]:
n_organisms = 256 #number of organisms
n_elites = 16 #number of elite organisms
n_generations = 2 #number of generations

config = {'n_organisms': n_organisms,
          'n_elites': n_elites,
          'n_generations': n_generations,
          
          'crossover_rate': 1.0,
          'mutation_rate': 1.0,
          'gamma': 0.05,
         }

In [3]:
output_folder_name = 'NEW_NAME'
output_folder_path = '../../results/'

config['output_folder_name'] = os.path.join(output_folder_path, output_folder_name)

## Processing data and c-typesmodel

Fitness function is declared in ga/mpi_scripts/loss_utils.py

Currently, RMSE of the signal, signal derivative or their weighted superposition is supported.

In [4]:
config['loss'] = 'RMSE'

## Data headers which GA should use 

Input CSV data files of protocols, recorded patch-clamp traces and output of class InaModel from ./../ga/mpi_scripts/ina_model.py are expected to have column headers for data.

In our model protocol header has name *'v'*, input recorded data from patch-clamp have header *'I_out'* and 
output of c-types model has header *'I_out'* too.

In [5]:
config['columns_control'] = ['I_out']
config['columns_model'] = ['I_out']
config['column_stim_protocol'] = ['v']

## Paths to protocols and trace

In [6]:
trace_dir = '../data/traces/'
protocol_dir = '../data/protocols/'
initial_state_protocol_dir = '../data/initial_protocols/'

In [7]:
### Traces
traces = pd.DataFrame(os.listdir(trace_dir), columns=['filename'])
traces.index.name ='n_trace'
traces

,filename
n_trace,
0,inactivation.csv
1,activation.csv


In [8]:
### Protocols

protocols = pd.DataFrame(os.listdir(protocol_dir), columns=['filename'])
protocols.index.name ='n_protocol'
protocols

,filename
n_protocol,
0,inactivation.csv
1,activation.csv


In [9]:
### Voltage between sweeps, below it will be called initial state protocol

initial_state_protocols = pd.DataFrame(os.listdir(initial_state_protocol_dir), columns=['filename'])
initial_state_protocols.index.name ='n_trace'
initial_state_protocols

,filename
n_trace,
0,.~lock.activation_initial_state.csv#
1,activation_initial_state.csv
2,inactivation_initial_state.csv


In [10]:
# Above shown which traces, protocols and initial state protocols are saved in default directories
# You can choose which of them will be used by number

# number of trace
n_trace = 1
#number of protocol
n_protocol = 1
# number of initial_state_protocol
n_initial_state_protocol = 1

trace_path = os.path.abspath(os.path.join(trace_dir, traces.filename[n_trace]))
protocol_path = os.path.abspath(os.path.join(protocol_dir, protocols.filename[n_protocol]))
initial_state_protocol_path = os.path.abspath(os.path.join(initial_state_protocol_dir, 
                                           initial_state_protocols.filename[n_initial_state_protocol]))

individual = {}
individual['filename_phenotype'] = trace_path
individual['filename_protocol'] = protocol_path
individual['filename_initial_state_protocol'] = initial_state_protocol_path

## Model of current
The model of experimental setup which will be used in ga should be compiled as a C library.

In [11]:
dirname_model = '../src/model_ctypes/PC_model/'
filename_so = 'libina.so'

config['filename_so'] =  os.path.abspath(os.path.join(dirname_model, filename_so))
config['filename_legend_states'] =  os.path.abspath(os.path.join(dirname_model, 'legend_states.csv'))
config['filename_legend_constants'] =  os.path.abspath(os.path.join(dirname_model, 'legend_constants.csv'))
config['filename_legend_algebraic'] =  os.path.abspath(os.path.join(dirname_model, 'legend_algebraic.csv'))

In [12]:
config

{'n_organisms': 256,
 'n_elites': 16,
 'n_generations': 2,
 'crossover_rate': 1.0,
 'mutation_rate': 1.0,
 'gamma': 0.05,
 'output_folder_name': '../../results/NEW_NAME',
 'loss': 'RMSE',
 'columns_control': ['I_out'],
 'columns_model': ['I_out'],
 'column_stim_protocol': ['v'],
 'filename_so': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/libina.so',
 'filename_legend_states': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_states.csv',
 'filename_legend_constants': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_constants.csv',
 'filename_legend_algebraic': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_algebraic.csv'}

## Free parameters and their boundaries

In [13]:
config['experimental_conditions'] = {}

In *common* part written all experimental setup parameters, in *individual* part - parameters of sodium current  and paths to initial state protocol, protocol and trace.

In [14]:
individual['params'] = {'c_p': {'bounds': [0.01,5.0], 'is_multiplier': True},
                   'c_m': {'bounds': [0.1, 5.0],  'is_multiplier': True},
                   'R': {'bounds': [0.1, 100.0], 'is_multiplier': True},
                   'R_f': {'bounds': [0.01, 50.0], 'is_multiplier': True},
                   'g_max': {'bounds': [0.05, 100.0], 'is_multiplier': True},
                   'g_leak': {'bounds': [0.1, 10.0], 'is_multiplier': True},
                   'x_c_comp': {'bounds': [0.1, 100.0], 'is_multiplier': True},
                   'x_r_comp': {'bounds': [0.1, 10.0], 'is_multiplier': True},
                   'alpha': {'bounds': [0.72, 0.78], 'is_multiplier': False},
                   'v_rev': {'bounds': [16.0, 80.0], 'is_multiplier': False},
                   'v_off': {'bounds': [-2.0, 2.0], 'is_multiplier': False},
                   'tau_z': {'bounds': [0.4, 6.0], 'is_multiplier': True}}

In [15]:
default_settings = {'bounds': [0.1, 10], 'is_multiplier': True}

common = {}
common_parameters = ['a0_m','b0_m','delta_m','s_m', 'tau_m_const',
                    'a0_h', 'b0_h','delta_h','s_h', 'tau_h_const',
                    'a0_j', 'b0_j', 'delta_j', 's_j', 'tau_j_const',
                    ]


# Using default settings
common['params'] = {param: default_settings for param in common_parameters}

# You can change bounds for free parameters if it's necessary
common['params']['tau_m_const']['bounds'] =  [0.05, 2.0]



Below prepared parameters bounds, which we used in original publication 

In [16]:
common['params']['v_half_h'] = {'bounds': [60.0, 80.0], 'is_multiplier': False}
common['params']['v_half_m'] = {'bounds': [20.0, 30.0], 'is_multiplier': False}
common['params']['k_h'] = {'bounds': [6.0, 15.0], 'is_multiplier': False}
common['params']['k_m'] = {'bounds': [6.0, 15.0], 'is_multiplier': False}


In [17]:
config['experimental_conditions']['common'] = common
config['experimental_conditions']['individual'] = individual

In [18]:
config

{'n_organisms': 256,
 'n_elites': 16,
 'n_generations': 2,
 'crossover_rate': 1.0,
 'mutation_rate': 1.0,
 'gamma': 0.05,
 'output_folder_name': '../../results/NEW_NAME',
 'loss': 'RMSE',
 'columns_control': ['I_out'],
 'columns_model': ['I_out'],
 'column_stim_protocol': ['v'],
 'filename_so': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/libina.so',
 'filename_legend_states': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_states.csv',
 'filename_legend_constants': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_constants.csv',
 'filename_legend_algebraic': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_algebraic.csv',
 'experimental_conditions': {'common': {'params': {'a0_m': {'bounds': [0.05,
      2.0],
     'is_multiplier': True},
    'b0_m': {'bounds': [0.05, 2.0], 'is_multiplier': True},
    'delta_m': {'bounds': [0.05, 2.0], 'is_multiplier': True},
    's_m': {'bounds': [0.05, 2.0], 'is_multiplier': True},

## Save config

In [19]:
# import json

# config_name = 'NEW_NAME'
# with open(f"../ga/configs/{config_name}.json", 'w') as f:
#     f.write(json.dumps(config, indent=4))

## Advanced 
Simultaneous optimization of multiple experimental traces is supported. In this case  'common' parameters such as ionic current properties are supposed to be the same in every experiment, while 'individual' such as experimental artifact parameters are supposed to be different. 

In [20]:
import copy
advanced_config = copy.deepcopy(config)
advanced_config['experimental_conditions']['individual_1'] = individual

In [21]:
# import json

# advanced_config_name = 'advanced_config'
# with open(f"../ga/configs/{advanced_config_name}.json", 'w') as f:
#     f.write(json.dumps(advanced_config, indent=4))